# Computer Programming 1105

In [2]:
# Ignore this; my laptop is so old and Apple is nasty so I need this
sdkroot = !xcrun --show-sdk-path
%env SDKROOT={sdkroot[0]}

env: SDKROOT=/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk


## Template theory and practice

Template is a way to write codes that has unknown types
but we have a very clear idea of what behavior we want.

Pair of the same type

```C++
template <typename T1>
struct Pair_same {
    T1 first;
    T1 second;
    Pair_same(const T1& a, const T1& b) : first(a), second(b) {} // constructor
    T1& operator[](int i) {
        if (i) return second;
        else return first;
    }
    const T1 operator[](int i) const {
        if (i) return second;
        else return first;
    }
};
```

Usage
```C++
Pair_same<int> p1(10, 20); // call constructor
Pair_same<int> p2 = {30, 40}; // similar to array initialization
cout << p1[0] << ", " << p1[1] << endl; // use [] to access
p1.first = 50; // modify first directly
p2[1] = 60; // use [] to modify second (because reference is returned)
```

The reason we need two `operator[];` is due to the following dilemma

- If we want to modify the value like `p2[1] = 60;`
  we need to return a reference.
- We need a const version to allow read-only access.
  This is because in `operator+`, we usually don't modify the operand.
  The keyword `const` in `operator+(const T other)`
  tells the compiler that we **promise** not to modify `other`.
  The compiler treat our promise very serious.
  So if in `operator+` we call the non-const version of `operator[]`,
  the compiler will complain.

There is no way we can achieve both in only one definition.

The std's `pair` is the with different types

```C++
template <typename T1, typename T2>
struct Pair_diff {
    T1 first;
    T2 second;
    Pair_diff(const T1& a, const T2& b) : first(a), second(b) {} // constructor
    T1& operator[](int i) {
        if (i) return second;
        else return first;
    }
    const T1 operator[](int i) const {
        if (i) return second;
        else return first;
    }
};

To define tuples of arbitrary length,
we need a feature of C++ called **variadic** templates/functions.

Some example of variadic functions like
`sum, max, min, average, printf, ...`

They are very hard to use and teach.
So we just use template containers.

## Container templates

C++ std library defines an array template
where the size is part of the template parameters.

```C++
template <typename T1, int size>
struct array {
    T1 elems[size];
    T1& operator[](int i) { return elems[i]; }
    const T1& operator[](int i) const { return elems[i]; }
};
```

Usages
```C++
array<int, 5> B;
B[0] = 10;
B[1] = 20;
cout << B[0] << ", " << B[1] << endl;
```

Now, when we defines functions, we can extract the size
as part of the template parameters.

```C++
template <typename T, int size>
sum(T array<T, size>& B, int size) {
    T total = B[0];
    for (int i = 1; i < size; ++i) {
        total += B[i];
    }
    return total;
}

```

The vector template is perhaps more useful than array
because re can resize it.

This means that the size is no longer a template parameter.
You need to call a getter function to learn the size.

```C++
template <typename T1>
struct vector {
    T1* elems;
    int size;
    T1& operator[](int i) { return elems[i]; }
    int size() { return size; }
    vector(int s) : size(s) {
        elems = new T1[s];
    }
    void resize(int new_size) {
        T1* new_elems = new T1[new_size];
        int min_size = (new_size < size) ? new_size : size;
        for (int i = 0; i < min_size; i++)
            new_elems[i] = elems[i];
        delete[] elems;
        elems = new_elems;
        size = new_size;
    }
    T1 sum() {
        T1 total = elems[0];
        for (int i = 1; i < size; ++i) {
            total += elems[i];
        }
        return total;
    }
};
```

Or you can define `sum` outside of the `struct`:

```C++
template <typename T1>
T1 sum(vector<T1> a) {
    T1 total = a[0];
    for (int i = 1; i < a.size(); i++) {
        total += a[i];
    }
    return total;
}
```

## Doubley-linked list

```C++
std::list<int> fibo = {2, 3, 5, 13, 21};

auto b = fibo.begin();
fibo.insert(++ ++ ++b,  8);

for (auto f : fibo) {
    cout << f;
}
```

## Double-ended queue

```C++
std::deque<int> fibo = {3, 5, 8, 13};
    
fibo.push_front(2);
fibo.push_back(21);

for (auto f : fibo) {
    cout << f;
}

fibo.pop_front();
fibo.pop_back();
```

BTW, all containers support the `for auto` syntax.

```C++
for (auto x : B) {
    f(x);
}
```

is roughly equivalent to

```C++
for (auto it = B.begin(); it != B.end(); ++it) {
    f(*it);
}
```

In other words, the `for auto` syntax
works with any container that has `begin()` and `end()` methods.


## Priority queue

```C++
struct HW {
    string subject;
    int daysLeft;
    bool operator< (HW other) const {
        return daysLeft > other.daysLeft;
    }
};
std::priority_queue<HW> HwToDo;
HwToDo.push({"Linear Algebra", 5});
HwToDo.push({"Da 1 English", 3});
HwToDo.push({"Computer Prog", 7});

cout << "do this first" << HWToDo.top().subject;
```

```C++
std::set<string> playlist;
playlist.insert("To lightyear and beyond");
playlist.insert("A balloon called white");
playlist.insert("Are you OK?");
playlist.insert("Actor");
playlist.insert("A balloon called white");

cout << "top songs" << endl;
for (auto song : playlist) {
    cout << song << endl;
} // only four lines
```

```C++
std::multiset<string> playlist;
playlist.insert("To lightyear and beyond");
playlist.insert("A balloon called white");
playlist.insert("Are you OK?");
playlist.insert("Actor");
playlist.insert("A balloon called white");

cout << "top songs" << endl;
for (auto song : playlist) {
    cout << song << endl;
} // five lines
```

## Map (python dictionary)

```C++
std::map<std::string, int> city_population;

city_population["New York"] = 8419600;
city_population["Los Angeles"] = 3980400;
city_population["Chicago"] = 2716000;
city_population.insert(
    std::Pair{"Houston", 2328000});

cout << "Population of New York: "
     << city_population["New York"] << endl;

for (auto pair : city_population) {
    cout << "City: " << pair.first
         << ", Population: " << pair.second << endl;
}
```

## Some words on iterators

iterator = how you are gonna visit things.

Forward iterator:
An iterator that lets you move forward
by overloading operator++;

Bidirectional iterator:
An iterator that lets you move both ways
by overloading both ++ and --.

random-access iterators:
An iterator that lets you [i]
by overloading operator[].


```C++
slink<int> L; // a singly-linked list
auto b = L.begin();
b++; // moving forward
cout << *b;

link<int> L; // a doubly-linked list
auto b = L.begin();
b++; // moving forward
b--; // moving backward
cout << *b;
```

For forward and bidirectional iterator;
If you want to find something,
Check everything.

```C++
auto find(T x) {
    for (auto p = L.begin(); p != L.end(); p++){
        if (*p == x) return p;
    }
    return p;
}
```

If you want to be able to use binary search, your iterator must support random access.

## find or check criteria

```C++
auto is11x = [](int n) {
    return n % 11 == 0;
}

std::find_if(fibo.begin(),fibo.end(), is11x);
std::count_if(fibo.begin(),fibo.end(), is11x);
```

## sortin


```C++
auto comparison_func = [](int a, int b) {
    if (a % 11 != b % 11) return a % 11 < b % 11;
    return a < b;
}
std::sort(fibo.begin(), fibo.end(), comparison_func);
```

## Container operations

```C++
std::vector<int> vec = {1, 2, 3, 4};
std::for_each(
    vec.begin(),
    vec.end(),
    [](int &n) { n *= n; }
);
// vec = {1, 4, 9, 16}
```

```C++
bool allEven = std::all_of(
    vec.begin(),
    vec.end(),
    [](int n) { return n % 2 == 0; }
);

bool allOdd = std::none_of(
    vec.begin(),
    vec.end(),
    [](int n) { return n % 2 == 0; }
);

bool containEven = std::any_of(
    vec.begin(),
    vec.end(),
    [](int n) { return n % 2 == 0; }
);
```

```C++
std::accumulate(
    numbers.begin(),
    numbers.end(),
    1,
    multiply
);

std::accumulate(
    numbers.begin(),
    numbers.end(),
    0,
    add
);
```

## Span


```C++
// g++ -std=c++20 demo.cpp
#include <span>
#include <vector>
#include <array>
#include <iostream>

int sum(std::span<const int> s) {
    int total = 0;
    for (int x : s) total += x;
    return total;
}

int main() {
    std::array<int, 5> a{10, 20, 30, 40, 50};
    std::vector<int> v{1, 2, 3, 4, 5};
    int c_arr[] = {5, 7, 8, 9, 10};

    std::span<int> whole_vec(v);           // view over vector storage
    std::span<int> sub = whole_vec.subspan(1, 3); // [2, 3, 4]
    std::cout << sum(sub) << '\n';         // 9

    std::cout << sum(a) << '\n';           // 150
    std::cout << sum(c_arr) << '\n';       // 39

    sub[0] = 42;                           // modifies v[1]
    std::cout << v[1] << '\n';             // 42
}
```

## Optional and expected

```C++
std::optional<int> opt;
// opt.reset();
opt = 42;
if (opt.has_value())
    cout << "Value: " << opt.value() << endl;
else
    cout << "No value." << endl;
```


```C++

std::expected<int, std::string> safe_div(int a, int b) {
    if (b == 0) return std::unexpected("division by zero");
    return a / b;
}
```

## Variante and Any

```C++
std::variant<int, std::string> v;   // initially holds int{0}
v = 42;
std::cout << std::get<int>(v) << "\n";

v = "hello";
std::cout << std::get<std::string>(v) << "\n";
```

```C++
std::any X;

X = 42;
cout << std::any_cast<int>(X) << endl;
X = string("Hello, any");
cout << std::any_cast<string>(X) << endl;
```

## printf, nice to see you again

```C++
std::print("Default: {}\n", PI); // 3.141592653589793
std::print("Fixed:   {:.2f}\n", PI); // 3.14
std::print("Sci:     {:e}\n", PI); // 3.14e+00
std::print("Percent: {:.1%}\n", PI); // 314.2%
std::print("Dec: {}\n", 42); // 42
std::print("Bin: {:b}\n", 42); // 101010
std::print("Hex: {:X}\n", 42); // 2A
std::print("Prefixed: {:#x}\n", 42); // 0x2a
std::print("|{:>5}|\n", 42); // |   42|
std::print("|{:<5}|\n", 42); // |42   |
std::print("|{:05}|\n", 42); // |00042|
std::print("{:’}\n", 1234567); // 1,234,567
std::print("|{:>10.2f}|\n", x);  // right align, width 10, 2 decimals
std::print("|{:<10.2f}|\n", x);  // left align
std::print("|{:^10.2f}|\n", x);  // center align
std::print("|{:*>10.2f}|\n", x); // right align, pad with '*'
```

## Practices from last year

Complete the definitions of Polynomial and Matrix and overload related functions properly.

```C++

int main() {
    srand(time(NULL));
    Matrix<int, 2, 2> A;
    A[0][0] = rand() % 100;
    A[0][1] = rand() % 100;
    A[1][0] = rand() % 100;
    A[1][1] = rand() % 100;
    cout << "Matrix A is" << endl;
    cout << A << endl;
    Polynomial<int, 2> poly = A.char_poly();
    cout << "The characteristic polynomial of A is" << endl;
    cout << poly('x') << endl;
    cout << "P(-1) = " << poly(-1) << endl;
    cout << "P(0) = " << poly(0) << endl;
    cout << "P(1) = " << poly(1) << endl;
    cout << "P(A) is " << endl;
    cout << poly(A) << endl;
}


```

Finish the following program to test if your computer can do multi-threading.

```C++
#include <iostream>
#include <thread>
#include <chrono>
#include <mutex>
using namespace std;

std::mutex OnlyOneThreadCanRun;

// This is the "main" function for the B sound
void Bfunc() {
    std::this_thread::sleep_for(std::chrono::milliseconds(?));
    while (true) {
        OnlyOneThreadCanRun.lock();
        cout << "B" << endl;
        OnlyOneThreadCanRun.unlock();
        std::this_thread::sleep_for(std::chrono::milliseconds(?));
    }
}

// This is the "main" function for the TS sound
void TSfunc() {
    string indent = "          ";
    while(true) {
        OnlyOneThreadCanRun.lock();
        cout << indent << "TS" << endl;
        OnlyOneThreadCanRun.unlock();
        std::this_thread::sleep_for(std::chrono::milliseconds(?));
    }
}

// This is the "main" function for the K sound
void Kfunc() {
    std::this_thread::sleep_for(std::chrono::milliseconds(?));
    string indent = "                    ";
    while (true) {
        OnlyOneThreadCanRun.lock();
        cout << indent << "K" << endl;
        OnlyOneThreadCanRun.unlock();
        std::this_thread::sleep_for(std::chrono::milliseconds(?));
    }
}

// This is the mainest main function, the main function of all
int main() {
    std::thread Bthread(Bfunc);
    std::thread TSthread(TSfunc);
    std::thread Kthread(Kfunc);

    Bthread.join();
    TSthread.join();
    Kthread.join();

    cout << "All threads join the main thread.  Music ends." << endl;
}
```